In [1]:
from simpletransformers.question_answering import QuestionAnsweringModel
from nested_lookup import nested_lookup
import pandas as pd
import numpy as np
import json
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
nltk.download("punkt")
import string
import shutil
import datetime as dt
import torch
import random
import itertools
import sys, os
import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package stopwords to /home/ilker/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/ilker/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
with open('train-v0.1.json' , encoding='utf-8') as json_file: 
    data = json.load(json_file)
    
data_list = list()
for i in data['data']:
    for j in i['paragraphs']:
        for k in j['qas']:
            k['is_impossible'] = False
            for z in k['answers']:
                tmp = z['answer_start']
                z['answer_start'] = int(tmp)
        data_list.append(j)

## training

In [3]:
def get_model(model_name):
    model_base = model_name.split("/")[1].split("-")[0]
    model = QuestionAnsweringModel(model_base, model_name, use_cuda = True)
    return model

## make predictions

In [4]:
def soru_sor(soru,id, context,model):
    soru = [{
        'context': context,

        'qas': [
            {
                'question':soru,
                'id':id 
            }
        ]
    }]
    
    return model.predict(soru)

In [5]:
def test_data_input(input_file):
    # Context into list
    f = open(input_file,"r",encoding = "utf-8")
    first_line = f.readline()
    del first_line
    # context
    context_list =    []
    context_line = True
    for line in f:
        if not line.strip():continue
        if line.startswith("context"):
            context_line = True
            a_list = []
        elif line.startswith("topic"):
            lines = ' '.join(a_list)
            context_list.append(lines)
            context_line = False
        if context_line:
            a_list.append(line.rstrip("\n").lstrip("\ufeff").strip("|").lstrip("context:").lstrip(": ").lstrip(" "))
    # questions into list
    f = open(input_file,"r",encoding = "utf-8")
    question_list =    []
    question_line = True
    for line in f:
        if not line.strip():continue
        if line.startswith("question"):
            question_line = True
            a = []
        elif line.startswith("answer"):
            lines = ' '.join(a)    
            question_list.append(lines)
            question_line = False
        else:continue
        if question_line:
            a.append(line.rstrip("\n").lstrip("\ufeff").strip("|").lstrip("question:").lstrip(" ").rstrip(" "))
    # id into list
    f = open(input_file,"r",encoding = "utf-8")
    id_list =    []
    id_line = True
    for line in f:
        if not line.strip():continue
        if line.startswith("id"):
            id_line = True
        else:continue
        if id_line:
            id_list.append(line.rstrip("\n").lstrip("id:").strip("|").strip(" "))

    # To find number of questions for each topic
    f = open(input_file,"r",encoding = "utf-8")
    topic = []
    topic_line = True
    for line in f:
        if not line.strip():continue
        if line.startswith("topic"):
            topic_line = True
            a = []
        elif line.startswith("question"):
            lines = ' '.join(a)
            topic.append(lines)
            topic_line = False
        else:continue
        if topic_line:
            a.append(line.rstrip("\n").lstrip("topic:").rstrip("|").rstrip(" ").strip(" "))
    # answers
    f = open(input_file,"r",encoding = "utf-8")
    answer_list =    []
    answer_line = True
    for line in f:
        if not line.strip():continue
        if line.startswith("answer"):
            answer_line = True
            a = []
        elif line.startswith("id"):
            lines = ' '.join(a)    
            answer_list.append(lines)
            answer_line = False
        else:continue
        if answer_line:
            a.append(line.rstrip("\n").strip("|").lstrip("answer:").lstrip(" ").rstrip(" "))    
    # make contexts as much as multiplied by their questions
    topic_counter_dict = {}
    topic_counts = []
    for top in topic:
        if not top in topic_counter_dict.keys():
            topic_counter_dict[top] = 1
        elif top in topic_counter_dict.keys():
            topic_counter_dict[top] += 1
    for key,value in topic_counter_dict.items():
        topic_counts.append(value)
    a = [[con]*topic_counts[i] for i,con in enumerate(context_list)]
    context_list_with_num = []
    for context_list in a:
        for context_repeat in context_list:
            context_list_with_num.append(context_repeat)
    return context_list_with_num, question_list, id_list, topic, answer_list

In [6]:
def remove_stopwords(some_list):
    for token in some_list:
        if token in stopwords.words("turkish"):
            del some_list[some_list.index(token)]
    return some_list

In [7]:
def exact_match(predicted, actual):
    matched_answers = []
    guessed_true = 0
    for i, answer in enumerate(actual):
        if predicted[i].lower().translate(str.maketrans('', '', string.punctuation)).strip(" ") == actual[i].lower().translate(str.maketrans('', '', string.punctuation)).strip(" "):
            guessed_true += 1 
            matched_answers.append(predicted[i])
    return guessed_true,matched_answers

# prepare data

In [8]:
with open('train-v0.1.json' , encoding='utf-8') as json_file: 
    data = json.load(json_file)
    
data_list = list()
for i in data['data']:
    for j in i['paragraphs']:
        for k in j['qas']:
            k['is_impossible'] = False
            for z in k['answers']:
                tmp = z['answer_start']
                z['answer_start'] = int(tmp)
        data_list.append(j)

In [9]:
len(data_list)

2232

# models

# all models

### cv

In [10]:
def get_model(model_name, params):
    model_base = model_name.split("/")[1].split("-")[0]
    model = QuestionAnsweringModel(model_base, model_name, use_cuda = True, args = params)
    return model

In [11]:
def make_cv_data(data_list):

    cv_context_list = []
    cv_question_list = []
    cv_topic = []
    cv_answer_list = []
    cv_id_list = []

    for list_idx in range(len(data_list)):
        tmp_context = data_list[list_idx]["context"]
        for i in range(len(data_list[list_idx]["qas"])):
            tmp_question = data_list[list_idx]["qas"][i]["question"]
            tmp_answer = data_list[list_idx]["qas"][i]["answers"]


            cv_context_list.append(tmp_context)
            cv_question_list.append(tmp_question)
            cv_answer_list.append(tmp_answer)
            
    for i in range(1, len(cv_answer_list) +1):
        cv_id_list.append(str(i).zfill(4))
        
    return cv_context_list, cv_question_list, cv_answer_list, cv_id_list

def partition(lst, n, random_state):
    random.seed(random_state)
    random.shuffle(lst) 
    division = len(lst) / float(n) 
    return [ lst[int(round(division * i)): int(round(division * (i + 1)))] for i in range(n) ]

In [12]:
train_args = dict(num_train_epochs = [1,2,3],train_batch_size = [1,2,3], max_seq_length = [64, 128, 192])
keys = train_args.keys()
values = (train_args[key] for key in keys)
combinations = [dict(zip(keys, combination)) for combination in itertools.product(*values)]

In [13]:
model = None

In [14]:
def pretty_size(size):
    """Pretty prints a torch.Size object"""
    assert(isinstance(size, torch.Size))
    return " × ".join(map(str, size))

def dump_tensors(gpu_only=True):
    """Prints a list of the Tensors being tracked by the garbage collector."""
    import gc
    total_size = 0
    for obj in gc.get_objects():
        try:
            if torch.is_tensor(obj):
                if not gpu_only or obj.is_cuda:
                    print("%s:%s%s %s" % (type(obj).__name__, 
                                          " GPU" if obj.is_cuda else "",
                                          " pinned" if obj.is_pinned else "",
                                          pretty_size(obj.size())))
                    total_size += obj.numel()
            elif hasattr(obj, "data") and torch.is_tensor(obj.data):
                if not gpu_only or obj.is_cuda:
                    print("%s → %s:%s%s%s%s %s" % (type(obj).__name__, 
                                                   type(obj.data).__name__, 
                                                   " GPU" if obj.is_cuda else "",
                                                   " pinned" if obj.data.is_pinned else "",
                                                   " grad" if obj.requires_grad else "", 
                                                   " volatile" if obj.volatile else "",
                                                   pretty_size(obj.data.size())))
                    total_size += obj.data.numel()
        except Exception as e:
            pass        
    print("Total size:", total_size)

In [15]:
def blockPrint():
    sys.stdout = open(os.devnull, 'w')

# Restore
def enablePrint():
    sys.stdout = sys.__stdout__

In [ ]:
["dbmdz/bert-base-turkish-cased"
,"dbmdz/bert-base-turkish-128k-uncased"
,"savasy/bert-base-turkish-sentiment-cased"
,"dbmdz/bert-base-turkish-uncased"
,"savasy/bert-base-turkish-squad"
,"dbmdz/bert-base-turkish-128k-cased"
,"savasy/bert-turkish-text-classification"
,"dbmdz/electra-base-turkish-cased-discriminator"
,"savasy/bert-base-turkish-ner-cased"
,"lserinol/bert-turkish-question-answering"
,"dbmdz/distilbert-base-turkish-cased"
,"dbmdz/electra-small-turkish-cased-discriminator"
,"erayyildiz/electra-turkish-cased"
,"dbmdz/electra-base-turkish-cased-generator"
,"savasy/bert-turkish-uncased-qnli"
,"dbmdz/electra-base-turkish-cased-v0-discriminator"
,"dbmdz/electra-small-turkish-cased-generator"
,"dbmdz/electra-base-turkish-cased-v0-generator"]

In [16]:
pretrained_models = ["dbmdz/bert-base-turkish-128k-cased",
"dbmdz/bert-base-turkish-128k-uncased",
"dbmdz/bert-base-turkish-cased",
"dbmdz/bert-base-turkish-uncased",
"dbmdz/distilbert-base-turkish-cased",
"dbmdz/electra-base-turkish-cased-discriminator",
"dbmdz/electra-small-turkish-cased-discriminator"]

sets = partition(data_list, 5, 42)
model_name = pretrained_models[0]
split_size = 5


cv_params = []
cv_split = []
cv_predictions = []
cv_answers = []
cv_duration = []

for cv in range(split_size):
    

    train_set = [x for i,x in enumerate(sets) if i!=cv]
    train_set = [item for sublist in train_set for item in sublist]
    test_set = sets[cv]
    cv_context_list, cv_question_list, cv_answer_list, cv_id_list = make_cv_data(test_set)
          
    for comb in combinations:
        
        t1 = dt.datetime.now()
        
        dirs = !ls
        dirs = pd.Series(dirs)
        dels = dirs[dirs.isin(["cache_dir","outputs","runs"])].to_list()

        for folder_name in dels:
            try:
                shutil.rmtree(folder_name)
            except:
                pass
        
        blockPrint()
        
        model = get_model(pretrained_models[0], comb)
        model.train_model(train_set, verbose = False)
        #context_list_with_num, question_list, id_list, topic, answer_list = test_data_input("test_questions_compiled.txt")

        predicted_answers = []

        for i in range(len(cv_context_list)):
            answerNprobs = (soru_sor(cv_question_list[i], cv_id_list[i], cv_context_list[i], model))
            answer = answerNprobs[0][0]["answer"][0]
            predicted_answers.append(answer)
            
        t2 = dt.datetime.now()
            
        cv_params.append(comb)
        cv_split.append(cv)
        cv_predictions.append(predicted_answers)
        cv_answers.append(cv_answer_list)
        cv_duration.append(t2-t1)
        
        
        del model
        del answerNprobs
        del answer
        del predicted_answers

        torch.cuda.empty_cache()
        
        enablePrint()
        
        print(cv)
        print(comb)

add example index and unique id: 100%|██████████| 6672/6672 [00:00<00:00, 986234.23it/s]


add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 11305.40it/s]


add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 4593.98it/s]


add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 9341.43it/s]


add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 13315.25it/s]


...

add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 9198.04it/s]


add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 9467.95it/s]


add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 11949.58it/s]


add example index and unique id: 100%|██████████| 6672/6672 [00:00<00:00, 1082359.17it/s]


RuntimeError: CUDA out of memory. Tried to allocate 376.00 MiB (GPU 0; 7.79 GiB total capacity; 5.97 GiB already allocated; 228.69 MiB free; 6.15 GiB reserved in total by PyTorch)